# Build Simple CNN + MLP -> MLP Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.nn import weighted_cross_entropy_with_logits

2024-04-10 16:59:30.639610: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 16:59:30.662427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 16:59:30.662447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 16:59:30.662927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 16:59:30.666577: I tensorflow/core/platform/cpu_feature_guar

In [2]:
image_path = "processed_data/partially_processed/uncompressed"
batch_size = 32
img_height = 128
img_width = 128
epochs = 10

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train_table_df = pd.read_pickle("processed_data/partially_processed/top500_df_elevation_onehot_grouped.pkl")
train_image_df_npz = np.load("processed_data/partially_processed/top500_images.npz")
train_image_df = train_image_df_npz['arr_0']

In [5]:
train_table_df.head()

,surveyId,Unnamed: 0,lon,lat,year,areaInM2,region,speciesId,ALPINE,ATLANTIC,BLACK SEA,BOREAL,CONTINENTAL,MEDITERRANEAN,PANNONIAN,STEPPIC,51,53,96,129,140,146,167,169,171,249,254,262,300,340,391,394,398,423,424,433,469,476,478,509,540,544,559,581,593,623,643,649,651,661,694,786,791,799,822,843,868,890,896,907,958,960,963,976,981,982,1015,1018,1020,1037,1038,1051,1085,1086,1092,1110,1131,1139,1162,1170,1180,1214,1254,1264,1276,1288,1387,1421,1424,1455,1472,1497,1498,1525,1531,1539,1545,1654,1677,1703,1705,1707,1712,1715,1716,1736,1757,1761,1793,1818,1858,1888,1889,1910,1951,1964,1986,2009,2013,2025,2101,2109,2126,2142,2145,2163,2189,2211,2250,2262,2308,2386,2398,2421,2474,2482,2490,2556,2570,2587,2608,2621,2634,2684,2703,2747,2753,2783,2788,2813,2823,2840,2847,2885,2891,2908,2922,2949,2955,2961,3020,3037,3043,3067,3092,3096,3123,3125,3140,3151,3161,3177,3214,3227,3236,3251,3275,3294,3318,3347,3358,3361,3375,3427,3438,3447,3451,3457,3476,3488,3497,3581,3639,3722,3738,3792,3803,3838,3879,3885,3935,3954,3958,3969,4003,4027,4077,4078,4102,4238,4279,4341,4349,4397,4404,4447,4483,4492,4493,4498,4499,4508,4513,4530,4541,4598,4609,4628,4638,4659,4671,4707,4734,4735,4748,4758,4854,4871,4898,5018,5022,5040,5071,5114,5115,5149,5166,5173,5189,5285,5335,5374,5383,5384,5386,5398,5400,5412,5420,5474,5542,5543,5557,5618,5704,5737,5739,5745,5760,5771,5773,5804,5829,5850,5868,5900,5901,5928,5980,5994,6002,6039,6044,6056,6079,6091,6097,6118,6119,6153,6183,6208,6235,6310,6331,6358,6377,6437,6491,6551,6571,6625,6634,6643,6666,6684,6685,6686,6707,6746,6788,6836,6848,6874,6883,6900,6903,6909,6914,6915,6925,6950,6962,6964,6989,6999,7040,7047,7052,7094,7121,7122,7207,7227,7244,7315,7318,7322,7333,7371,7395,7450,7536,7559,7566,7588,7613,7617,7644,7661,7670,7679,7687,7706,7739,7748,7755,7760,7788,7790,7807,7808,7817,7824,7862,7871,7880,7942,7957,7971,7972,7978,7990,7999,8106,8124,8149,8151,8208,8258,8284,8316,8422,8424,8428,8431,8439,8503,8508,8512,8519,8532,8542,8573,8632,8635,8639,8654,8705,8746,8747,8748,8760,8807,8818,8858,8904,8908,8917,8923,8926,8949,8983,9005,9024,9028,9069,9072,9076,9165,9190,9204,9207,9245,9261,9273,9311,9341,9368,9376,9388,9439,9458,9465,9493,9513,9548,9555,9572,9580,9600,9610,9630,9636,9647,9669,9674,9707,9710,9714,9771,9816,9873,9877,9881,9890,9892,9963,10005,10020,10024,10047,10051,10073,10082,10112,10113,10116,10218,10223,10247,10255,10268,10274,10281,10305,10315,10317,10364,10367,10385,10392,10427,10460,10474,10514,10520,10525,10569,10600,10631,10684,10762,10763,10881,10882,10892,10896,10899,10904,10909,10927,10940,10950,10969,10991,10998,11005,11054,11078,11120,11140,11157,11188,11193,11195
0,212,11,3.099038,43.134956,2021,100.00000,MEDITERRANEAN,11193,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,

In [6]:
Y = train_table_df.iloc[:,16:]

In [7]:
Y.head()

,51,53,96,129,140,146,167,169,171,249,254,262,300,340,391,394,398,423,424,433,469,476,478,509,540,544,559,581,593,623,643,649,651,661,694,786,791,799,822,843,868,890,896,907,958,960,963,976,981,982,1015,1018,1020,1037,1038,1051,1085,1086,1092,1110,1131,1139,1162,1170,1180,1214,1254,1264,1276,1288,1387,1421,1424,1455,1472,1497,1498,1525,1531,1539,1545,1654,1677,1703,1705,1707,1712,1715,1716,1736,1757,1761,1793,1818,1858,1888,1889,1910,1951,1964,1986,2009,2013,2025,2101,2109,2126,2142,2145,2163,2189,2211,2250,2262,2308,2386,2398,2421,2474,2482,2490,2556,2570,2587,2608,2621,2634,2684,2703,2747,2753,2783,2788,2813,2823,2840,2847,2885,2891,2908,2922,2949,2955,2961,3020,3037,3043,3067,3092,3096,3123,3125,3140,3151,3161,3177,3214,3227,3236,3251,3275,3294,3318,3347,3358,3361,3375,3427,3438,3447,3451,3457,3476,3488,3497,3581,3639,3722,3738,3792,3803,3838,3879,3885,3935,3954,3958,3969,4003,4027,4077,4078,4102,4238,4279,4341,4349,4397,4404,4447,4483,4492,4493,4498,4499,4508,4513,4530,4541,4598,4609,4628,4638,4659,4671,4707,4734,4735,4748,4758,4854,4871,4898,5018,5022,5040,5071,5114,5115,5149,5166,5173,5189,5285,5335,5374,5383,5384,5386,5398,5400,5412,5420,5474,5542,5543,5557,5618,5704,5737,5739,5745,5760,5771,5773,5804,5829,5850,5868,5900,5901,5928,5980,5994,6002,6039,6044,6056,6079,6091,6097,6118,6119,6153,6183,6208,6235,6310,6331,6358,6377,6437,6491,6551,6571,6625,6634,6643,6666,6684,6685,6686,6707,6746,6788,6836,6848,6874,6883,6900,6903,6909,6914,6915,6925,6950,6962,6964,6989,6999,7040,7047,7052,7094,7121,7122,7207,7227,7244,7315,7318,7322,7333,7371,7395,7450,7536,7559,7566,7588,7613,7617,7644,7661,7670,7679,7687,7706,7739,7748,7755,7760,7788,7790,7807,7808,7817,7824,7862,7871,7880,7942,7957,7971,7972,7978,7990,7999,8106,8124,8149,8151,8208,8258,8284,8316,8422,8424,8428,8431,8439,8503,8508,8512,8519,8532,8542,8573,8632,8635,8639,8654,8705,8746,8747,8748,8760,8807,8818,8858,8904,8908,8917,8923,8926,8949,8983,9005,9024,9028,9069,9072,9076,9165,9190,9204,9207,9245,9261,9273,9311,9341,9368,9376,9388,9439,9458,9465,9493,9513,9548,9555,9572,9580,9600,9610,9630,9636,9647,9669,9674,9707,9710,9714,9771,9816,9873,9877,9881,9890,9892,9963,10005,10020,10024,10047,10051,10073,10082,10112,10113,10116,10218,10223,10247,10255,10268,10274,10281,10305,10315,10317,10364,10367,10385,10392,10427,10460,10474,10514,10520,10525,10569,10600,10631,10684,10762,10763,10881,10882,10892,10896,10899,10904,10909,10927,10940,10950,10969,10991,10998,11005,11054,11078,11120,11140,11157,11188,11193,11195
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [8]:
X_train_image, X_val_image, Y_train, Y_val = train_test_split(train_image_df, Y, test_size=0.2, shuffle=True, random_state=113)

In [9]:
# print shapes of X_train_table, X_val_table, X_train_image, X_image_image, Y_train, Y_val
print(X_train_image.shape)
print(X_val_image.shape)
print(Y_train.shape)
print(Y_val.shape)


(70340, 128, 128, 3)
(17586, 128, 128, 3)
(70340, 500)
(17586, 500)


## Image Only Model

In [10]:
num_classes = Y_val.shape[1]

In [11]:
normalisation_layer = layers.Rescaling(1./255)

In [12]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


2024-04-10 17:00:22.913814: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 17:00:22.937832: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 17:00:22.937979: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        1

In [15]:
cp = ModelCheckpoint('model/top500/', save_best_only=True)

In [16]:
history = model.fit(x=X_train_image, y=Y_train, validation_data=(X_val_image, Y_val), epochs=epochs, callbacks=[cp])

Epoch 1/10


2024-04-10 17:00:54.190790: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-04-10 17:00:55.196174: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6823a89c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-10 17:00:55.196193: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-10 17:00:55.198889: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712761255.245248   49916 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2199/2199 [==============================] - ETA: 0s - loss: 0.1866 - accuracy: 0.1020INFO:tensorflow:Assets written to: model/top500/assets


INFO:tensorflow:Assets written to: model/top500/assets


2199/2199 [==============================] - 12s 4ms/step - loss: 0.1866 - accuracy: 0.1020 - val_loss: 0.1682 - val_accuracy: 0.0740
Epoch 2/10
2199/2199 [==============================] - 8s 4ms/step - loss: 0.1671 - accuracy: 0.0697 - val_loss: 0.1689 - val_accuracy: 0.0803
Epoch 3/10
2199/2199 [==============================] - 8s 4ms/step - loss: 0.1685 - accuracy: 0.0810 - val_loss: 0.1695 - val_accuracy: 0.0803
Epoch 4/10
2199/2199 [==============================] - 8s 4ms/step - loss: 0.1692 - accuracy: 0.0810 - val_loss: 0.1698 - val_accuracy: 0.0803
Epoch 5/10
2199/2199 [==============================] - 8s 4ms/step - loss: 0.1694 - accuracy: 0.0810 - val_loss: 0.1701 - val_accuracy: 0.0803
Epoch 6/10
2199/2199 [==============================] - 8s 4ms/step - loss: 0.1703 - accuracy: 0.0810 - val_loss: 0.1714 - val_accuracy: 0.0803
Epoch 7/10
2199/2199 [==============================] - 8s 4ms/step - loss: 0.1710 - accuracy: 0.0810 - val_loss: 0.1719 - val_accuracy: 0.0803
Ep

### Combined Model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Define the Picture (CNN) Stream
input_pic = layers.Input(shape=(128, 128, 6))
x         = layers.Conv2D(128, (6, 6), activation='relu', input_shape=(128, 128, 6))(input_pic)
x         = layers.MaxPooling2D((2, 2))(x)
x         = layers.Conv2D(64, (3, 3), activation='relu')(x)
x         = layers.MaxPooling2D((2, 2))(x)
x         = layers.Conv2D(64, (3, 3), activation='relu')(x)
x         = layers.MaxPooling2D((5, 5))(x)
x         = layers.Flatten()(x)
x         = layers.Dense(200, activation='relu')(x)
x         = Model(inputs=input_pic, outputs=x)

# Define the Stats (Feed-Forward) Stream
input_stats = layers.Input(shape=(42,))
y = layers.Dense(128, activation="relu")(input_stats)
y = layers.Dense(200, activation="relu")(y)
y = Model(inputs=input_stats, outputs=y)

# Concatenate the two streams together
combined = layers.concatenate([x.output, y.output])

# Define joined Feed-Forward Layer
z = layers.Dense(400, activation="relu")(combined)
z = layers.Dense(400, activation="relu")(z)


# Define output node of 5016 neurons (classification task)
z = layers.Dense(5016, activation="softmax")(z)


# Define the final model
model = Model(inputs=[x.input, y.input], outputs=z)

2024-04-08 17:57:24.488483: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 17:57:24.568787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 17:57:24.568813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 17:57:24.586776: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 17:57:24.614316: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 6)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 123, 123, 128)        27776     ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 61, 61, 128)          0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 59, 59, 64)           73792     ['max_pooling2d[0][0]'] 

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Create a model saving callback and train for 10 epochs (connect to GPU runtime!!)

from tensorflow.keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint('model/', save_best_only=True)

model.fit(x=[X_train_image, X_train_table], y=Y_train, validation_data=([X_val_image, X_val_table], Y_val), epochs=10, callbacks=[cp])

Epoch 1/10


2024-04-08 17:57:32.459338: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-04-08 17:57:32.832414: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f43713fd820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-08 17:57:32.832432: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-08 17:57:32.848365: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712591852.972999   36405 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2225/2225 [==============================] - 36s 15ms/step - loss: nan - accuracy: 0.0039 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 2/10
2225/2225 [==============================] - 34s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 3/10
2225/2225 [==============================] - 34s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 4/10
2225/2225 [==============================] - 33s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 5/10
2225/2225 [==============================] - 31s 14ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 6/10
2225/2225 [==============================] - 33s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 7/10
2225/2225 [==============================] - 32s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_